## Imports
---
Before we begin, we import the top-level necessary packages. Moreover, we include the algorithm specific packages from the local directories.

In [ ]:
# Get the system environment variables
import os
import pickle
import sys

from IPython import display


# General Imports
import numpy as np
from time import time

# Variational
from auxiliary import collect_obs
from auxiliary import initialize_Ab0
from core import smoothing

# Plotting
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Dynamic
from dynamics import sys_lorenz_63_ut as dynamics

%matplotlib inline

# Change the default figure size.
plt.rcParams['figure.figsize'] = (10.0, 8.0)

## Simulation for Lorenz 63

In [ ]:
# Show current python version.
print(" >> Python version is: {0}\n".format(sys.version))

'''
TIME-WINDOW PARAMETERS:
'''
# Initial, final and time step.
t0 = 0
tf = 20
dt = 0.01

# Define the time-window of inference.
Tw = np.arange(t0, tf + dt, dt)

# Number of discretized points.
N = Tw.shape[0]

'''
SYSTEM SPECIFIC PARAMETERS:
'''
# Dimensionality of the system.
D = 3

# Stochastic Noise (variance).
sigma_Noise = 10 * np.eye(D)

# Drift parameters (sigma, rho, beta).
theta_Drift = np.array([10, 28, 8. / 3])

# Observation Noise (variance).
obs_Noise = 2 * np.eye(D)

# Print a message.
print("System tested: {0}, T=[{1},{2}]\n".format('Lorenz 63',t0,tf))

'''
OBSERVATIONS SECTION:
'''
# Define the observation density (# of observations per time unit).
n_Obs = 5

# We need at least one observation (per time unit)
n_Obs = np.max([n_Obs, 1])

# Observation operator: np.eye(D)
H = np.eye(D)

# Create the (artificial) true trajectory.
xt_true = dynamics.system_path(Tw, sigma_Noise, theta_Drift)

# Sample the noisy observations from the true path.
obsX, obsY = collect_obs.collect_obs(xt_true, Tw, n_Obs, obs_Noise)

dynamics.plot_sample_path(xt_true)

dynamics.plot_sample_path(obsY)

'''
PRIOR MOMEMTS:
'''
# Prior moment of initial condition noise variance.
# p(x0) ~ N(mu, tau0)
tau0 = 0.5 * np.eye(D)

# Get the true sample value at time t=0
prior_x0 = {'mu0': xt_true[0], 'tau0': tau0}

# Initial mean m(t=0)
m0 = xt_true[0] + 0.1 * np.random.randn(1)

# Initial covariance matrix S(t=0): K*np.eye(D)
S0 = 0.25 * np.eye(D)


'''
PACKING PARAMETERS:
'''
# ODE solver: {'Euler', 'Heun', 'RK2', 'RK4'}
ode_method = 'Euler'

# Create a dictionary to hold all the model parameters.
sde_struct = {
    'Sig': sigma_Noise, 
    'theta': theta_Drift, 
    'Rig': obs_Noise,
    'D': D, 
    'H': H, 
    'obsX': obsX,
    'obsY': obsY,
    'px0': prior_x0,
    'Tw': Tw, 
    'dt': dt, 
    'N': N, 
    'ode_method': ode_method,
    'checkGradf': False
}

'''
INITIALIZATION:
'''
# Maximum number of iterations.
nit = 500

# Generate initial variational parameters (initial search point).
Ab0 = initialize_Ab0.initialize_Ab0(S0, sde_struct)

# Main Operation.
print(' [VGPA] (Smoothing) Experiment in progress. Please wait ...')

# Start the timer.
tic = time()

# Full Variational approximation.
Fmin, mParam = smoothing.smoothing(dynamics.energy_mode, Ab0, m0, S0, sde_struct, nit)

# Stop the timer.
ttime = time() - tic

# Display termination message.
print(' [VGPA] (Smoothing) Experiment ended in {0:.2f} seconds.'.format(ttime))

# Display the minimum free energy.
print(' Minimum var. free energy is {0:.2f}.\n'.format(Fmin))

'''
PLOTTING SECTION:
'''
for i in range(3):
    # Extract here the quantities for ploting.
    mt = mParam['mt'][:, i] # marginal   mean   values.
    St = [mParam['St'][j, :, :][i, i] for j in range(N)] # marginal variance values.

    # Create a new figure.
    ax = plt.figure().add_subplot(3, 1, i + 1)

    # Prepare the error-bars.
    upper_line = mt + 2 * np.sqrt(St)
    lower_line = mt - 2 * np.sqrt(St)
    ax.fill_between(Tw, upper_line.flatten(), lower_line.flatten(), facecolor='lightblue')

    # Add the mean path: m(t) and the true sample path: x(t).
    ax.plot(Tw, mt, 'r-', linewidth=3, label='m(t)')
    ax.plot(Tw, xt_true[:, i], 'k--', linewidth=3, label='x(t)')
    ax.legend(loc='lower right')
    ax.grid(True)

    # Add the (noisy) observations.
    obsX = (np.array(obsX) + 1).tolist()
    ax.plot(Tw[obsX], obsY[:, i], 'o', markersize=8)

    # Setup the labels.
    ax.set_ylabel('m(t) +/- 2*std(t)', fontdict={'fontsize': 20, 'fontweight': 'bold'})
    ax.set_xlabel('time', fontdict={'fontsize': 20, 'fontweight': 'bold'})
    ax.set_title('VGPA - Simulation of Lorenz 63 state {}'.format(i + 1), fontdict={'fontsize': 20, 'fontweight': 'bold'})

    # Show-time.
    plt.show()

    # End-Of-File

In [ ]:
def get_file_name(base_dir, theta_Drift):
    return '{}/{:.2f}-{:.2f}-{:.2f}.pickle'.format(base_dir, theta_Drift[0], theta_Drift[1], theta_Drift[2])

In [ ]:
def generate_sample_path(base_dir):
    '''
    TIME-WINDOW PARAMETERS:
    '''
    # Initial, final and time step.
    t0 = 0
    tf = 20
    dt = 0.01

    # Define the time-window of inference.
    Tw = np.arange(t0, tf + dt, dt)

    # Number of discretized points.
    N = Tw.shape[0]

    '''
    SYSTEM SPECIFIC PARAMETERS:
    '''
    # Dimensionality of the system.
    D = 3

    # Stochastic Noise (variance).
    sigma_Noise = 10 * np.eye(D)

    # Drift parameters (sigma, rho, beta).
    theta_Drift = np.array([10, 28, 8. / 3])

    # Observation Noise (variance).
    obs_Noise = 2 * np.eye(D)

    # Define the observation density (# of observations per time unit).
    n_Obs = 5

    # We need at least one observation (per time unit)
    n_Obs = np.max([n_Obs, 1])

    # Observation operator: np.eye(D)
    H = np.eye(D)
    
    # Create the (artificial) true trajectory.
    xt_true = dynamics.system_path(Tw, sigma_Noise, theta_Drift)

    # Sample the noisy observations from the true path.
    obsX, obsY = collect_obs.collect_obs(xt_true, Tw, n_Obs, obs_Noise)
    
    data = {
        't0': t0,
        'tf': tf,
        'dt': dt,
        'Tw': Tw,
        'N': N,
        'D': D,
        'sigma_Noise': sigma_Noise,
        'theta_Drift': theta_Drift,
        'obs_Noise': obs_Noise,
        'n_Obs': n_Obs,
        'H': H,
        'xt_true': xt_true,
        'obsX': obsX,
        'obsY': obsY
    }
    
    file_name = get_file_name(base_dir, theta_Drift)
    
    with open(file_name, 'wb') as file:
        pickle.dump(data, file)
        
    return data

In [ ]:
def load_sample_path(base_dir, data):
    file_name = get_file_name(base_dir, data['theta_Drift'])
    with open(file_name, 'rb') as file:
        return pickle.load(file)

In [ ]:
def run_grid_search(base_dir, data):
    '''
    PRIOR MOMEMTS:
    '''
    # Prior moment of initial condition noise variance.
    # p(x0) ~ N(mu, tau0)
    tau0 = 0.5 * np.eye(data['D'])

    # Get the true sample value at time t=0
    prior_x0 = {'mu0': data['xt_true'][0], 'tau0': tau0}

    # Initial mean m(t=0)
    m0 = data['xt_true'][0] + 0.1 * np.random.randn(1)

    # Initial covariance matrix S(t=0): K*np.eye(D)
    S0 = 0.25 * np.eye(data['D'])


    '''
    PACKING PARAMETERS:
    '''
    # ODE solver: {'Euler', 'Heun', 'RK2', 'RK4'}
    ode_method = 'Euler'

    # Create a dictionary to hold all the model parameters.
    sde_struct = {
        'Sig': data['sigma_Noise'], 
        'theta': data['theta_Drift'], 
        'Rig': data['obs_Noise'],
        'D': data['D'], 
        'H': data['H'], 
        'obsX': data['obsX'],
        'obsY': data['obsY'],
        'px0': prior_x0,
        'Tw': data['Tw'], 
        'dt': data['dt'], 
        'N': data['N'], 
        'ode_method': ode_method,
        'checkGradf': False
    }

    '''
    INITIALIZATION:
    '''
    # Maximum number of iterations.
    nit = 500

    # Generate initial variational parameters (initial search point).
    Ab0 = initialize_Ab0.initialize_Ab0(S0, sde_struct)

    # Main Operation.
    print(' [VGPA] (Smoothing) Experiment in progress. Please wait ...')

    # Start the timer.
    tic = time()

    # Full Variational approximation.
    Fmin, mParam = smoothing.smoothing(dynamics.energy_mode, Ab0, m0, S0, sde_struct, nit)

    # Stop the timer.
    ttime = time() - tic
        
    result = {
        'theta_Drift': data['theta_Drift'],
        'Fmin': Fmin,
        'mParam': mParam,
        'ttime': ttime,
    }

    file_name = get_file_name(base_dir, data['theta_Drift'])
    with open(file_name, 'wb') as file:
        pickle.dump(result, file)

In [ ]:
base_dir = './lorenz-63-ut-results/'

data = generate_sample_path(base_dir)

for sigma in [6, 8, 10, 12, 14, 16]:
    for rho in [24, 26, 28, 30, 32, 34]:
        for beta in [1, 1.6, 2.2, 2.8, 3.4, 4]:
            test_data = load_sample_path(base_dir, data)            
            test_data['theta_Drift'] = np.array([sigma, rho, beta])
            run_grid_search(base_dir, test_data)
            display.clear_output()        

In [ ]:
%matplotlib notebook
plt.rcParams['figure.figsize'] = (10.0, 8.0)

theta_Drift_true = np.array([10, 28, 8. / 3])
sigmas = [6, 9, 12, 15]
rhos = [25, 28, 31, 34]
betas = [1, 2, 3, 4]
plot_grid_search_result('./lorenz-63-ut-results-1/', sigmas, rhos, betas)

In [ ]:
def plot_grid_search_result(base_dir, sigmas, rhos, betas):
    surface_data = []
    
    for sigma in sigmas:
        for rho in rhos:
            for beta in betas:
                file_name = os.path.join(base_dir, '{:.2f}-{:.2f}-{:.2f}.pickle'.format(sigma, rho, beta))
                with open(file_name, 'rb') as file:
                    datum = pickle.load(file)
                    surface_data.append([sigma, rho, beta, datum['f_min']])

    surface_data = np.array(surface_data)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    sp = ax.scatter(surface_data[:, 0], surface_data[:, 1], surface_data[:, 2], s=50, c=surface_data[:, 3],
                    cmap=cm.pink)
    plt.colorbar(sp)
    plt.show()